In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
DATADIR = './data/'

In [3]:
pstdf = pd.read_csv(DATADIR + 'pst_full_data.txt', sep='\t')
print(pstdf.head())

     Code subject         sleep  sr  BaselineFirst  female  age  night  \
0  S1-1-B      S1  Normal sleep   0              1       0   28    1.5   
1  S1-1-B      S1  Normal sleep   0              1       0   28    1.5   
2  S1-1-B      S1  Normal sleep   0              1       0   28    1.5   
3  S1-1-B      S1  Normal sleep   0              1       0   28    1.5   
4  S1-1-B      S1  Normal sleep   0              1       0   28    1.5   

   days_between_tests testtime  ... Actual wake time  Actual wake (%)  \
0                   7    13:51  ...              NaN              NaN   
1                   7    13:51  ...              NaN              NaN   
2                   7    13:51  ...              NaN              NaN   
3                   7    13:51  ...              NaN              NaN   
4                   7    13:51  ...              NaN              NaN   

  Sleep efficiency  Sleep latency get_up_easy well_rested KSS  SUSS kss_rt_ms  \
0              NaN            NaN  